# Generating Images using Generative Adverserial Network (GAN)

### 1. Introduction

Among other generative models, Generative Adverserial Networks (GANs) is my favorite. They are powerful yet simple. GANs consist of two neural networks called generator and discriminator. As the name suggest, these neural networks compete with each other. The generator is responsible for creating/generating image from a random set of numbers called latent vector. The discriminator is responsible for classifying these images to be either real or fake. 

<img src='gan_figure.png' style="width: 700px;"> </img>


Throughout training, the generator tries to fool discriminator by generating more and more realistic images in each epoch. At some point, the generator generates images so that the discriminator cannot distinguish whether they are real or fake.

### 2. Implementation

Let us import the required modules. I am going to use PyTorch library for constructing the network. The other modules are responsible for time tracking and visualization.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from time import time

import torch
from torch import nn
from torch.utils.data import DataLoader

from torchvision import datasets, transforms
from torchvision.utils import save_image

It is always a good practice to define the device that we will use for the rest of the calculations. The below code simply determines the available device which can be either CPU or GPU. We should always keep in mind that we have to keep all data on the same device otherwise it would cause conflict. Here we define the device once and send the relevant data to that device throughout the code.

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

We then need to download the dataset with an appropriate transformation. Here, we are going to only use the transform that transforms the data to PyTorch tensor.

In [ ]:
train_data = datasets.MNIST(root='../Data',  
                            train=True, 
                            download=True, 
                            transform=transforms.ToTensor()
                           )

Let us define the hyperparameters that will be used till the end of the calculations. It is more practical to define these parameters globally as we are doing here because it makes the network easier to tune. Otherwise one must go to every place where these parameters are defined. I used 100 training epochs. This means that the trainer passes through the dataset 100 times and uses 60 images for a batch. It is kind of a convention to make latent size 100 so I keep it that way. Lastly, lr, b1 and b2 are the parameters of the Adam optimizer. lr is the learning rate. b1 and b2 are the constant parameters of Adam.

In [ ]:
# Hyperparameters
hparam = {"num_epochs" : 100, # Number of training epochs
          "batch_size" : 60,
          "latent_size" : 100,  # Size of z latent vector (i.e. size of generator input)    
          "lr" : 0.0001,  # Learning rate for optimizer
          "b1" : 0.9,
          "b2" : 0.999,
         }

Here we load the data from the dataset we have downloaded. Dataloader automatically divide the data into batches. The following code shows the images from the dataloader. It is not crucial but it is always nice to check how the data looks like.

In [ ]:
dataloader = DataLoader(train_data, 
                        batch_size=hparam["batch_size"], 
                        shuffle=True)

In [ ]:
im, _ = next(iter(dataloader))
plt.imshow(im[0].numpy().reshape(28, 28), cmap="gray")
plt.show()

We came to the most straightforward and delicate part of the code: the implementation of the models. As mentioned in the introduction, we have two networks compete with each other. The generator takes the vector from the latent space and map it to the images. Thus, we need to start with the vector with the dimension 100x1. Remember that 100 is the latent space size. Then, we gradually increase the size of the hidden layers and finally map it to 28x28 which is the image size that we are dealing with. I used ReLU activation function. You can always change these as you wish. LeakyReLU is also good choice and commonly used in GANs because it avoids the vanishing gradient problem. Here our network is not too deep so we do not face these kind of problems. I also used batch normalization for each layer. This makes the network converge faster and more stable.

In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_size=hparam["latent_size"], batch_size=hparam["batch_size"]):
        super().__init__()
        
        self.batch_size = batch_size
        
        self.model = nn.Sequential(
            nn.Linear(latent_size, 128),
            nn.BatchNorm1d(128, 0.1),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256, 0.1),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512, 0.1),
            nn.ReLU(),
            nn.Linear(512, 28*28),
        )
        
    def forward(self, z):
        img = self.model(z)
        return img.view(self.batch_size, 1, 28, 28)

The discriminator has the mirror imaged architecture. It maps the images to labels. It starts with the array of size 28x28 and maps it to one dimensional vector which gives us the score of how realistic the image is. By the way, for our task, the model can easily be modified to get better performance by replacing the linear hidden layers with convolutional layers.

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, latent_size=hparam["latent_size"], batch_size=hparam["batch_size"]):
        super().__init__()
        
        self.batch_size = batch_size
        
        self.model = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.BatchNorm1d(512, 0.1),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256, 0.1),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128, 0.1),
            nn.ReLU(),              
            nn.Linear(128, 1),
            nn.Sigmoid()
        )
            
    def forward(self, z):
        img_flat = z.view(self.batch_size, -1)
        return self.model(img_flat)

Now let us instantiate the models we have defined above and send them into the device we already specified in the beginning.

In [ ]:
gen_net = Generator().to(device)
dis_net = Discriminator().to(device)

We should also initialize the weights in the networks.

In [ ]:
def init_weights(m):
    classname = m.__class__.__name__
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif type(m) == nn.BatchNorm1d:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

gen_net.apply(init_weights)
dis_net.apply(init_weights)

I will use the binary cross entropy loss.

In [ ]:
criterion = nn.BCELoss().to(device)

We now define the optimizers with the parameters we specified in the beginning of the code.

In [ ]:
gen_optimizer = torch.optim.Adam(gen_net.parameters(), lr=hparam["lr"], betas=(hparam["b1"], hparam["b2"]))
dis_optimizer = torch.optim.Adam(dis_net.parameters(), lr=hparam["lr"], betas=(hparam["b1"], hparam["b2"]))

All we need to do is to train the network. This is the most cumbersome part. I normally prefer to use PyTorch Lightning module to avoid this part (and I also suggest for you). But here, I would like to implement my own training steps.

We first start with defining true_label and false_label. For each image, we need to give a qualitative score for each image that is created by the generator. We give 1 for a realistic image and 0 for an unrealistic image. Since a batch contains 64 images, the true_label and false_label vectors are the dimensions of 64x1. torch.ones() function can create a vector consisting of 64 ones in one go. torch.zeros() is also similar.

Then we define t1 to track elapsed time and dis_losses and gen_losses lists to track the losses of each network.

We iterate through all data in the dataset. In each iteration, we define a latent vector z from a uniform distribution. The generator creates images from z and tries to fool the discriminator. In the beginning, the created images will be messy but, as the networks are being trained, the generator creates more and more realistic images. After some time, the discriminator cannot distinguish wheter it is a real or created image from the generator. gen_data = gen_net(z) create an image and generator_loss = criterion(dis_net(gen_data), true_label) calculates the loss of the generator. The meaning of this loss is important because it shows us how good our generator fools the discriminator. Thus we expect/want this loss to increase.


We now need to train the discriminator. The discriminator is trained simultaneously with the generator. There are two losses for the discriminator. I named them real_loss and fake_loss. real_loss is the loss when we feed the real images (the images from MNIST dataset). fake_loss is the loss when we feed the generated images from the generator. The discriminator loss is the total (or average) of these fake_loss and real_loss and we need to minimize this sum. One very crucial step is that (honestly I spent some time to figure this out) we need to detach gen_data (the image generated by the generator) before calculating the loss.

The rest of the code is for visualization. The first if statement is responsible for keeping track of the current number of epoch and losses. The other if statement is for saving the batch of images to see the improvement and make a video out of it. It saves the created image to a directory. I then create videos using FFmpeg by combining the images. You can watch the video that shows the result of this code at https://www.youtube.com/watch?v=uhzq5gBdwes&t=10s&ab_channel=UtkuCanbolat.

In [ ]:
true_label = torch.ones((hparam["batch_size"], 1), requires_grad=False).to(device)
false_label = torch.zeros((hparam["batch_size"], 1), requires_grad=False).to(device)

t1 = time()
dis_losses = []
gen_losses = []

k = 0
for epoch in range(hparam["num_epochs"]):
    for i, (data, _) in enumerate(dataloader):
        data = data.to(device)
        
        #  ----------------------------------
        #  Train Generator
        #  ----------------------------------

        gen_optimizer.zero_grad()

        # Sample noise
        z = torch.Tensor(np.random.uniform(0, 1, size=(hparam["batch_size"], hparam["latent_size"]))).to(device)
        
        
        # Generated data
        gen_data = gen_net(z)       
        generator_loss = criterion(dis_net(gen_data), true_label)
        generator_loss.backward(retain_graph=True)
        gen_optimizer.step()
     
        #  ----------------------------------
        #  Train Discriminator
        #  ----------------------------------

        dis_optimizer.zero_grad()

        real_loss = criterion(dis_net(data), true_label)
        fake_loss = criterion(dis_net(gen_data.detach()), false_label) # detach is extremely important
        discriminator_loss = (real_loss + fake_loss)

        discriminator_loss.backward()
        dis_optimizer.step()

        #  -----------------------------------------------------------------
        
        if i % 250 == 0:          
            print(
                "[Epoch %d/%d]\t [Batch %d/%d]\t [Dis loss: %f]\t [Gen loss: %f]"
                % (epoch, hparam["num_epochs"], i, len(dataloader) , discriminator_loss.item(), generator_loss.item()))
            
        
            dis_losses.append(discriminator_loss.item())
            gen_losses.append(generator_loss.item())
            
        if i % 250 == 0:
            save_image(gen_data.data[:36], "generated_images/im" + str(k) + ".png", nrow=6)
            
            
            fig, axs = plt.subplots(2)
            plt.subplots_adjust(hspace=1)
            
            xaxis = [250*i for i in range(k+1)]
            
            axs[0].plot(xaxis, np.array(dis_losses))
            axs[0].set_title("Discriminator Loss")
            axs[0].set_xlabel("Number of Iterations")
            axs[0].set_ylabel("Loss")
            axs[0].grid(True)

            axs[1].plot(xaxis, np.array(gen_losses))
            axs[1].set_title("Generator Loss")
            axs[1].set_xlabel("Number of Iterations")
            axs[1].set_ylabel("Loss")

            axs[1].grid(True)
            plt.savefig("loss_images/loss"+ str(k) +".png", dpi=300)
            
            
            fig.show(False)
            plt.close('all')
            
            k += 1
            
print("\nTraining Time: " + str(time()-t1))

### 3. Additional Part

The images created by the network are very small (28x28 pixels) to make a good video. Thus I wrote the following code to upscale all the images in a given directory. It simply opens the images and performs a tensor multiplication using np.kron() so it does not change anything but the size. n and m determine how many times they extend the axis x and y, respectively.

In [ ]:
import os
import numpy as np
from matplotlib import pyplot as plt

def upscaler(im_path, n=3, m=3):
    im = plt.imread(im_path)
    im = np.mean(im, axis=2)
    im = np.kron(im, np.ones((n, m)))
    plt.imsave(im_path, im, cmap='gray')
    
directory = 'image_data'
for filename in os.listdir(directory):
    upscaler(os.path.join(directory, filename))